## WELCOME TO WEEK 11

The Epic Finale Week

And

# WELCOME TO THE **M**ODEL **C**ONTEXT **P**ROTOCOL!

### Please note

There may be changes here from the video as I'm always making updates!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">To my Windows PC people - an important announcement</h2>
            <span style="color:#ff7800;">I have unpleasant news. There's a problem running MCP Servers on Windows PCs; Mac and Linux is fine. This is a known issue as of May 4th, 2025. I asked o3 with Deep Research to try to find workarounds; it <a href="https://chatgpt.com/share/6817bbc3-3d0c-8012-9b51-631842470628">confirmed the issue</a> and confirmed the workaround.<br/><br/>
            The workaround is a bit of a bore. It is to take advantage of "WSL", the Microsoft approach for running Linux on your PC. You'll need to carry out more setup instructions! But it's quick, and several students have confirmed that this works perfectly for them, then the Week 6 MCP labs work. Plus, WSL is actually a great way to build software on your Windows PC.<br/>
            The WSL Setup instructions are in the Setup folder, <a href="../setup/SETUP-WSL.md">in the file called SETUP-WSL.md here</a>. I do hope this only holds you up briefly - you should be back up and running quickly. Oh the joys of working with bleeding-edge technology!<br/><br/>
            With many thanks to students Markus, Abhi, Hui-Ling, and several others, for helping me work on it and confirming the fix.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [ ]:
load_dotenv(override=True)

### Let's use MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

Let's try the Fetch mcp-server that we looked at last week

### Special note - in case you updated the OpenAI Agents framework:

OpenAI Agents SDK recently upgraded their SDK and made a breaking change (thank you, OpenAI 😂). I haven't yet updated the version in this repo to the latest version. If you've upgraded the package yourself, then you might get an error about missing positional arguments. If you get that error, you'll need to replace `await server.list_tools()` with `await server.session.list_tools()` and change the last line from `fetch_tools` to `fetch_tools.tools`. And you'll need to make this change every time you get this error! The joys of working at the bleeding edge..

In [ ]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

## Extra installation step - if you don't have Node and Playwright on your computer

The next MCP tool uses node (the Javascript Server), and it needs you to have the command 'npx' installed on your computer.

Extra setup steps are here:

[Node and Playwright setup](../setup/SETUP-node.md)

## And now 2 more MCP servers - this time Javascript based, with node

### If the next cell hangs, see the important troubleshooting at the end of the SETUP-node instructions linked above.

In [ ]:

##THE BELOW LINE IS FOR MAC USERS. USERS EXECUTING THIS LINE IN WSL ON WINDOPWS should comment out this line and uncomment the line below:

playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

# v -------- For WINDOWS USERS ------ v
# Please ensure that you replace your username and node version accordingly /home/{username}/.nvm/versions/node/v22.19.0/bin:
#playwright_params = {"command": "npx", "args": ["@playwright/mcp@latest"], "env": {"PATH": "/home/{username}}/.nvm/versions/node/v22.19.0/bin:" + os.environ["PATH"]}}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools


NameError: name 'MCPServerStdio' is not defined

In [ ]:

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))

##THE BELOW LINE IS FOR MAC USERS. USERS EXECUTING THIS LINE IN WSL ON WINDOPWS should comment out this line and uncomment the line below:
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

# v -------- For WINDOWS USERS ------ v
# Please ensure that you replace your username and node version accordingly /home/{username}/.nvm/versions/node/v22.19.0/bin:
#files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path], "env": {"PATH": "/home/mnedelko/.nvm/versions/node/v22.19.0/bin:" + os.environ["PATH"]}}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools

### And now.. bring on the Agent with Tools!

In [ ]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""


async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4.1-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md")
            print(result.final_output)


### Check out the trace

https://platform.openai.com/traces

### Now take a look at some MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article:
https://huggingface.co/blog/Kseniase/mcp



